In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import nltk
from tqdm import tqdm
import re
import random
from random import shuffle
import sklearn
random.seed(777)

data = {'utterance_data':[],'emotion_label':[],'emotion':[],'prompt':[], 'utterance_data_combined':[],'utterance_id':[],"speaker_info":[],"speaker_utterance":[], "listener_utterance":[], "conv_id":[]}

ModuleNotFoundError: No module named 'nltk'

In [12]:
train_df = pd.read_csv('data/empathetic_dialogues/train.csv')
train_df.head(10)

,prompt,utterance_data,speaker_utterance,listener_utterance,emotion,emotion_label,speaker_info
0,I remember going to the fireworks with my best...,I remember going to see the fireworks with my ...,I remember going to see the fireworks with my ...,"Was this a friend you were in love with, or ju...",13,sentimental,"[0, 1, 0, 1, 0, 1]"
1,i used to scare for darkness,it feels like hitting to blank wall when i se...,it feels like hitting to blank wall when i se...,Oh ya? I don't really see howI do actually hit...,11,afraid,"[0, 1, 0, 1, 0, 1]"
2,I showed a guy how to run a good bead in weldi...,Hi how are you doing todaydoing good.. how abo...,"Hi how are you doing todayIm good, trying to u...",doing good.. how about youit's quite strange t...,29,proud,"[0, 1, 0, 1, 0]"
3,I have always been loyal to my wife.,I have never cheated on my wife.And thats some...,I have never cheated on my wife.Yea it hasn't ...,"And thats something you should never do, good ...",5,faithful,"[0, 1, 0, 1]"
4,A recent job interview that I had made me feel...,"Job interviews always make me sweat bullets, m...","Job interviews always make me sweat bullets, m...",Don't be nervous. Just be prepared.Yes but if ...,2,terrified,"[0, 1, 0, 1, 0]"
5,I am very happy to have been first over 300 st...,"Hi, this year, I was the first over 300 studen...","Hi, this year, I was the first over 300 studen...",Sounds great! So what's your major?Well please...,6,joyful,"[0, 1, 0, 1]"
6,I once lost my job and got mad.,I lost my job last year and got really angry.I...,I lost my job last year and got really angry.Y...,I am sorry to hear that. Did it happen out of ...,22,angry,"[0, 1, 0, 1]"
7,"One year during christmas, i did not get a sin...","During christmas a few years ago, I did not ge...","During christmas a few years ago, I did not ge...","Wow, that must be terrible, I cannot imagine, ...",0,sad,"[0, 1, 0, 1, 0]"
8,"I was at work, and my coworker was called into...","My coworker is allowed to work remotely, but I...","My coworker is allowed to work remotely, but I...","I work remotely, I wish that you could do some...",7,jealous,"[0, 1, 0, 1]"
9,I was home alone. I heard a loud noise come fr...,The other night I was alone and heard a nose c...,The other night I was alone and heard a nose c...,NaN,2,terrified,[0]


In [14]:
########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################

def random_deletion(words, p):

    #obviously, if there's only one word, don't delete it
    if len(words) <= 1:
        return words

    #randomly delete words with probability p
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    #if you end up deleting all words, just return a random word
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return new_words

def swap_word(new_words):
    if len(new_words) <= 1:
        return new_words
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words

In [15]:
def eda(sentence, alpha_rs=0.1, p_rd=0.1, num_aug=4):
    words = sentence.split(' ')
    words = [word for word in words if word is not '']
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug/2)+1

    #random swap
    if (alpha_rs > 0):
        n_rs = max(1, int(alpha_rs*num_words))
        for _ in range(num_new_per_technique):
            a_words = random_swap(words, n_rs)
            augmented_sentences.append(' '.join(a_words))

    #random delete
    if (p_rd > 0):
        for _ in range(num_new_per_technique):
            a_words = random_deletion(words, p_rd)
            augmented_sentences.append(' '.join(a_words))

    shuffle(augmented_sentences)

    #trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    #append the original sentence
    augmented_sentences.append(sentence)

    return augmented_sentences

In [17]:
# for i in range(len(train_df)):
num_aug = 2
EDA_train_df = pd.DataFrame(train_df)
for i in tqdm(range(len(train_df))):
    utterance_data = train_df['utterance_data'].iloc[i]
    augmented_sentences = eda(utterance_data, alpha_rs=0.05, p_rd=0.1, num_aug=num_aug)
    # print(augmented_sentences)
    # print(len(augmented_sentences))
    tmp_data = train_df.iloc[i].to_dict()
#     if i == 4083:
#         print(utterance_data)
#         print(len(augmented_sentences))
#         print(augmented_sentences)
    for j in range(num_aug):
        tmp_data['utterance_data'] = augmented_sentences[j]
        tmp_df = pd.DataFrame(tmp_data, index=[len(EDA_train_df)])
        EDA_train_df = EDA_train_df.append(tmp_df)

print(len(EDA_train_df))
df_shuffled=sklearn.utils.shuffle(EDA_train_df)
df_shuffled.to_csv("data/empathetic_dialogues/EDA_train2.csv",index=False)

100%|██████████| 19533/19533 [06:24<00:00, 50.84it/s]


58599


In [3]:
# df_shuffled=sklearn.utils.shuffle(EDA_train_df)
# df_shuffled.to_csv("data/empathetic_dialogues/EDA_train2.csv",index=False)

NameError: name 'EDA_train_df' is not defined